In [34]:
import pickle
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE, SMOTEN
import pprint as pp
import data_prep

with open("trials/m=dt-s=over-b=10,d=none.pickle", 'rb') as f:
    result:GridSearchCV = pickle.load(f)
# print(result.best_params_)
# print()
# pp.pprint(result.cv_results_)

pipe = result.best_estimator_

df_train, _ = data_prep.get_prepped_dataset(bins=10)
df_X = df_train.drop('Class', axis=1)
df_y = df_train['Class']
X = df_X.to_numpy()
y = df_y.to_numpy()
y = LabelEncoder().fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

categories = [list(df_train[col].cat.categories) for col in df_train.drop('Class', axis=1).columns]

# pipe = ImbPipeline([
#     ('smoten', SMOTEN(random_state=123, sampling_strategy='not majority')),
#     ('onehot', OneHotEncoder(categories=[['bin-0','bin-1','bin-2','bin-3','bin-4','bin-5','bin-6','bin-7','bin-8','bin-9'],
#                                         ['V1', 'V2', 'V3', 'V4', 'V5'],
#                                         ['bin-0','bin-1','bin-2','bin-3','bin-4','bin-5','bin-6','bin-7','bin-8','bin-9'],
#                                         ['V1', 'V10', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9'],
#                                         ['V1', 'V2', 'V3', 'V4', 'V5'],
#                                         ['V1', 'V2', 'V3', 'V4'],
#                                         ['V1', 'V2', 'V3', 'V4'],
#                                         ['bin-0', 'bin-1', 'bin-2', 'bin-3', 'bin-4', 'bin-5', 'bin-6', 'bin-7', 'bin-8', 'bin-9'],
#                                         ['V1', 'V2', 'V3'],
#                                         ['V1', 'V2', 'V3', 'V4'],
#                                         ['bin-0', 'bin-1', 'bin-2', 'bin-3', 'bin-4', 'bin-5', 'bin-6', 'bin-7', 'bin-8', 'bin-9'],
#                                         ['V1', 'V2', 'V3'],
#                                         ['bin-0','bin-1','bin-2','bin-3','bin-4','bin-5','bin-6','bin-7','bin-8','bin-9'],
#                                         [1, 2, 3, 4],
#                                         ['V1', 'V2', 'V3', 'V4', 'V5'],
#                                         ['V1', 'V2'],
#                                         [1, 2, 3, 4],
#                                         ['V1', 'V2', 'V3'],
#                                         [1, 2, 3, 4],
#                                         ['V1', 'V2', 'V3', 'V4'],
#                                         [1.0, 2.0]])),
#         ('model_dt', DecisionTreeClassifier(criterion='entropy', min_samples_leaf=10))
# ])

pipe = ImbPipeline([
    ('smoten', SMOTEN(random_state=123, sampling_strategy='not majority')),
    ('onehot', OneHotEncoder(categories=categories)),
    ('model_dt', DecisionTreeClassifier(criterion='entropy', min_samples_leaf=10))
    ])

score = accuracy_score(pipe.predict(X_val), y_val)
print(f"Score: {score}")

Score: 0.6722222222222223
